# Продажи магазина Amazon

### Данные для анализа

Таблица `Sales` содержит информацию о ежемесячных продажах магазина Amazon за 2023 год. 

- **month**: Номер месяца
- **quantity**: Количество проданных товаров за месяц

### 1. Напишите запрос, который извлекает из предложенной базы данных номера всех месяцев, а также указывает для каждого месяца среднее количество проданных товаров, учитывая лишь предыдущий, текущий и следующий месяцы. 


```sql
SELECT month,
       AVG(quantity) OVER (ORDER BY month ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) AS avg_quantity
FROM Sales;
```

### 2. Напишите запрос, который извлекает из предложенной базы данных всю информацию о продажах за каждый месяц, а также указывает для каждого месяца количество проданных товаров за предыдущий месяц.



```sql
SELECT month,
       quantity,
       IFNULL(SUM(quantity) OVER (ORDER BY month ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING), 0) AS prev_month_quantity
FROM Sales;
```

### 3. Напишите запрос, который извлекает из предложенной базы данных всю информацию о продажах за каждый месяц, а также указывает для каждого месяца суммарное количество проданных товаров за текущий и все предыдущие месяцы.



```sql
SELECT month,
       quantity,
       SUM(quantity) OVER (ORDER BY month ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS total_quantity
FROM Sales;
```

### 4. Напишите запрос, который извлекает из предложенной базы данных всю информацию о продажах за каждый месяц, начиная с третьего, а также указывает для каждого месяца среднее количество проданных товаров за два предыдущих месяца.

 

```sql
SELECT month,
       quantity,
       AVG(quantity) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING) AS two_prev_months_avg_quantity
FROM Sales
LIMIT 100
OFFSET 2;
```

### 5. Напишите запрос, который извлекает из предложенной базы данных всю информацию о продажах за каждый месяц, а также указывает для каждого месяца суммарное количество проданных товаров за текущий и все предыдущие месяцы в рамках квартала.



```sql
WITH Quarter AS (
    SELECT month,
           quantity,
           NTILE(4) OVER (ORDER BY month) AS quarter
    FROM Sales
)

SELECT month,
       quantity,
       SUM(quantity) OVER (PARTITION BY quarter
                           ORDER BY month
                           ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS total_quantity_within_quarter
FROM Quarter;
```

### 6. Напишите запрос, который извлекает из предложенной базы данных всю информацию, а также указывает для каждого дня (даты), было ли продано в этот день товаров больше, чем в предыдущий день:

- yes, если товаров было продано больше
- no, если товаров было продано столько же или меньше
- unknown, если информация о продажах за предыдущий день отсутствует

```sql
SELECT
    day,
    quantity,
    CASE
        WHEN quantity > SUM(quantity) OVER query_days THEN 'yes'
        WHEN quantity <= SUM(quantity) OVER query_days THEN 'no'
        ELSE 'unknown'
    END AS sales_better_than_yesterday
FROM Sales
WINDOW query_days AS (ORDER BY day RANGE BETWEEN INTERVAL 1 DAY PRECEDING AND INTERVAL 1 DAY PRECEDING);
```

### 7. Напишите запрос, который извлекает из предложенной базы данных всю информацию о продажах за каждый месяц, а также указывает для каждого месяца номер месяца, в который было продано наибольшее количество товаров. Записи в результирующей таблице должны быть расположены в порядке возрастания значения поля month.



```sql
SELECT month,
       quantity,
       FIRST_VALUE(month) OVER (ORDER BY quantity DESC) AS highest_sales_month
FROM Sales
ORDER BY month;
```

### 8. Напишите запрос, который извлекает из предложенной базы данных всю информацию о продажах за каждый месяц, а также указывает для каждого месяца номер месяца, в который было продано наибольшее количество товаров в рамках квартала. Помимо номера месяца, должно быть указано само количество проданных товаров.

 

```sql
WITH Quarter_sales AS(
    SELECT month,
           quantity,
           NTILE(4) OVER (ORDER BY month) AS quarter
    FROM Sales)

SELECT month,
       quantity,
       FIRST_VALUE(month) OVER (PARTITION BY quarter ORDER BY quantity DESC) AS highest_sales_month_within_quarter,
       FIRST_VALUE(quantity) OVER (PARTITION BY quarter ORDER BY quantity DESC) AS highest_sales_quantity_within_quarter
FROM Quarter_sales
ORDER BY month
```

### 9. Напишите запрос, который разбивает все месяцы по кварталам, определяет для каждого квартала месяц, в который было продано наименьшее количество товаров, и отображает полученный результат в виде таблицы из трех полей:

- quarter — номер квартала
- lowest_sales_month — номер месяца, в который было продано наименьшее количество товаров
- lowest_sales_quantity — соответствующее месяцу количество проданных товаров

```sql
WITH Quarter_sales AS(
    SELECT month,
           quantity,
           NTILE(4) OVER (ORDER BY month) AS quarter
    FROM Sales)

SELECT DISTINCT quarter,
       FIRST_VALUE(month) OVER (PARTITION BY quarter ORDER BY quantity) AS lowest_sales_month,
       FIRST_VALUE(quantity) OVER (PARTITION BY quarter ORDER BY quantity) AS lowest_sales_quantity
FROM Quarter_sales
```

### 10. Напишите запрос, который извлекает из предложенной базы данных всю информацию о продажах за каждый месяц, а также указывает для каждого месяца количество проданных товаров за предыдущий и предпредыдущий месяцы.



```sql
SELECT month,
       quantity,
       LAG(quantity, 1, 0) OVER (ORDER BY month) AS prev_month_sales,
       LAG(quantity, 2, 0) OVER (ORDER BY month) AS second_prev_month_sales
FROM Sales;
```

### 11. Напишите запрос, который извлекает из предложенной базы данных всю информацию о продажах за каждый месяц, а также указывает для каждого месяца количество проданных товаров в соответствующем месяце предыдущего квартала: для первого месяца квартала соответствующим месяцем является первый месяц предыдущего квартала, для второго — второй, для третьего — третий.



```sql
SELECT Sales.*,
       LAG(quantity, 3, 0) OVER (ORDER BY month) AS prev_quarter_month_sales
FROM Sales;
```

### 12. Напишите запрос, который извлекает из предложенной базы данных всю информацию о продажах за те месяцы, в которые товаров было продано больше чем в предыдущем месяце.



```sql
WITH PrevMonth AS (
    SELECT month, quantity,
           LAG(quantity, 1, 0) OVER (ORDER BY month) AS prev_month_quantity
    FROM Sales
)
SELECT month, quantity
FROM PrevMonth
WHERE quantity > prev_month_quantity;
```

### 13. Напишите запрос, который извлекает из предложенной базы данных всю информацию о продажах за все месяцы, кроме последнего, а также указывает для каждого месяца разницу в количестве проданных товаров между текущим месяцем и следующим.



```sql
WITH FolowMonth AS (
    SELECT month, quantity,
           LEAD(quantity, 1) OVER (ORDER BY month) AS folow_month_quantity
    FROM Sales
    
)
SELECT month, 
       quantity,
       ABS(quantity - folow_month_quantity) AS next_month_sales_diff
FROM FolowMonth
LIMIT 11;
```

### 14. Интернет-магазин Amazon в рамках каждого квартала определяет среднее изменение количества проданных товаров, складывая разницу между продажами первого и второго месяцев квартала с разницей между продажами второго и третьего месяцев квартала и деля ее пополам. Напишите запрос, который разбивает все месяцы по кварталам, определяет среднее изменение количества проданных товаров в каждом квартале и отображает полученный результат в виде таблицы из двух полей:

- quarter — номер квартала
- sales_avg_diff — среднее изменение количества проданных товаров


```sql
WITH QuarterSales AS (
    SELECT 
        month,
        quantity,
        NTILE(4) OVER (ORDER BY month) AS quarter
    FROM 
        Sales
)

SELECT 
    quarter,
    (
        ABS(current_quantity - next_quantity) + 
        ABS(next_quantity - next_next_quantity)
    ) / 2 AS sales_avg_diff
FROM (
    SELECT 
        month,
        quarter,
        LEAD(quantity, 1) OVER (PARTITION BY quarter ORDER BY month) AS next_quantity,
        LEAD(quantity, 2) OVER (PARTITION BY quarter ORDER BY month) AS next_next_quantity,
        quantity AS current_quantity
    FROM 
        QuarterSales
) AS subquery
WHERE (
        (current_quantity - next_quantity) + 
        (next_quantity - next_next_quantity)
    ) / 2  IS NOT NULL

```